# BIO-SELECT - Marigliano
## Global pipeline

_TODO_ : insert global pipeline image here

## Imports

In [ ]:
from sklearn import neighbors, datasets
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

import itertools

%matplotlib inline

# set float precision at 2 digits
np.set_printoptions(precision=2)

## Constants

In [ ]:
N_FEATURES_ALGORITHM = 1000

## Dataset loading
_TODO_: 
* this notebook must only load one dataset
* retrieve dataset to load from cmd arguments or from env variable

In [ ]:
from datasets.EGEOD22619.EGEOD22619Dataset import EGEOD22619Dataset
from datasets.MILE.MileDataset import MileDataset
from datasets.Golub99.GolubDataset import GolubDataset

from datasets.DatasetEncoder import DatasetEncoder
from datasets.DatasetSplitter import DatasetSplitter

#ds = MileDataset()
#ds = EGEOD22619Dataset()
ds = GolubDataset()

## Dataset transformation
The dataset needs some transformations such as encoding the outputs as float (necessary for scikit learn), normalization, ...

_TODO_:
* dataset splitting (train, test[, validation])
* encode outputs
* normalization
* classes merging
    * due to the low class balancing we might want to regroup them. Example Healthy vs Non-Healthy (choose the most represented class ?)

In [ ]:
# encode Dataset string classes into numbers
ds_encoder = DatasetEncoder(ds)
ds = ds_encoder.encode()
ds = DatasetSplitter(ds, test_size=0.4)

X = ds.get_X()
y = ds.get_y()
print(len(y))

X_train = ds.get_X_train()
y_train = ds.get_y_train()
X_test = ds.get_X_test()
y_test = ds.get_y_test()

print(len(X_train[0]))

## Algorithms
Run the chosen algorithms and save them and their output subset of features using cPickle into files. They can be used later to display some graphs and to be analyzed

_TODO_: Write a subsection for each algorithm :
* OneVsRest or OneVsOne ?
    * only for those who needs it
* Grid search + CV
    * maybe not for all algorithms such as SVM RFE which takes a lot of time
    * not for algorthms which does not have parameters to tune (ReliefF, Fisher Score,...)
* print classification report (accuracy, recall, precision, ...)
    * issue: not all algortihms are able to do this
* normalize score using minmax normalization (0-1)
* show score per features (50 to 100 first ones)
* save algorithm in a file

Algorithms:
* ExtraTrees
* Random Forest
* SVM
* SVM RFE
* ANN
* ReliefF
* Fisher Score
* "Best features subset ~ SVM"
* SVM Backward ?
* CFS - Correlation-based Feature Selection
* Mutual Information Classifier
* One genetic based algorithm

In [ ]:
from algorithms.ExtraTreesAlgorithm import ExtraTreesAlgorithm
from algorithms.ReliefFAlgorithm import ReliefFAlgorithm
from algorithms.FisherScoreAlgorithm import FisherScoreAlgorithm
from algorithms.FValueAlgorithm import FValueAlgorithm
from algorithms.SVMAlgorithm import SVMAlgorithm

eta_grid = [{'n_estimators': np.arange(10, 1000, 300), 'criterion': ["gini", "entropy"], 'max_features': ["sqrt", "auto", "log2"], "n_jobs": [-1]}]
%time eta = ExtraTreesAlgorithm(ds, eta_grid)
print("ExtraTrees best params \n\t%s" % eta.best_params)


rff = ReliefFAlgorithm(ds)
fsa = FisherScoreAlgorithm(ds)
fva = FValueAlgorithm(ds)

#FIXME: grid search does not seem to work (for SVM at least)
gridsearch_params = [{
        'kernel':['linear'],
        'C':[200, 0.1, 1, 10, 100, 1000],
        'gamma' : [1e-2, 1e-3, 1e-4, 1e-5],
        'tol' : [1e-2, 1e-3, 1e-4, 1e-5],
        'cache_size':[1024]
    }]
%time svm_gs = SVMAlgorithm(ds, gridsearch_params)
print("Best params \n\t%s" % svm_gs.best_params)

%time svm = SVMAlgorithm(ds)

algorithms = [eta, rff, fsa, fva, svm_gs, svm]

subsets = []
alg_names = []
for alg in algorithms:
    feats_by_score = alg.get_best_features_by_score(n=N_FEATURES_ALGORITHM)
    
    subsets.append(feats_by_score)
    alg_names.append(alg.name)


## Features subsets merging
Each algorithm has done its work and provide a subset of features as:
* a ranked score list
* a ranked list (no score)
* a list (no ranking, no score)

This part uses some techniques to combine/merge theses lists into a better one

_TODO_: 
* Visualize the lists
    * Venn diagram ? --> limited to 3 sets, does not scale
    * matrix: show the similarity of features between two subsets
        * Jaccard
        * Union
        * Cosine similarity
* implement merge techniques
    * votation
    * weighted votation
    * union of intersection
    * ...

### Subsets visualization

In [ ]:
from scipy import spatial

# some set similarity functions
def intersection_count(a, b):
    return len(a.intersection(b))

def jaccard(a, b):
    return len(a.intersection(b))/float(len(a.union(b)))

def cosine_similarity(a, b):
    return 1.0 - spatial.distance.cosine(np.array(list(a)), np.array(list(b)))

def compute_similary_between_subsets(subsets, compare_func):
    N_subsets = len(subsets)
    similarity_matrix = np.zeros(shape=(N_subsets, N_subsets))

    for i, j in itertools.product(range(N_subsets), range(N_subsets)):
        subset_i = {i[0] for i in subsets[i]}
        subset_j = {j[0] for j in subsets[j]}
        similarity_matrix[i, j] = compare_func(subset_i, subset_j)
        
    return similarity_matrix

def plot_feature_subsets_matrix(cm, alg_names, title, cmap=plt.cm.Blues):
    title += "\n" # add a little margin for the title
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(alg_names))
    plt.xticks(tick_marks, alg_names, rotation=45)
    plt.yticks(tick_marks, alg_names)

    thresh = cm.max() / 2.0 + 0.1
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        text = "%.2f" % cm[i, j]
        plt.text(j, i, text,
                 horizontalalignment="center",
                 backgroundcolor="white",
                 #color="white" if cm[i, j] > thresh else "black")
                 color="black")

    plt.tight_layout()


similarity_matrix = compute_similary_between_subsets(subsets, compare_func=jaccard)
plt.figure(figsize=(6, 8))
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Jaccard similarity between two feature subsets")

similarity_matrix = compute_similary_between_subsets(subsets, compare_func=intersection_count)
plt.figure(figsize=(6, 8))
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Intersection between two feature subsets")

similarity_matrix = compute_similary_between_subsets(subsets, compare_func=cosine_similarity)
plt.figure(figsize=(6, 8))
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Cosine similarity between two features subsets")

### Subsets merging

In [ ]:
from merge.simple.SimpleUnionSubsetMerger import SimpleUnionSubsetMerger

susm = SimpleUnionSubsetMerger(subsets)
ensemble_set = susm.merge()

print("Unique features (union of all subsets): %d over a total of %d " % (len(ensemble_set), (N_FEATURES_ALGORITHM * len(subsets))))

## Evaluation of the merged subset
Once we have a merged list containing the best features, we would like to evaluate it with several classifiers

_TODO_: use a separate test set ? -> split again train/test set -> no changes in the Dataset class